In [ ]:
import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

import os
import glob
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

AUTOTUNE = tf.data.AUTOTUNE

In [44]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 1024
IMG_HEIGHT = 1024

In [45]:
def read_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image.set_shape([None, None, 3])
    # image = tf.image.resize(image, [1024, 1024],
    #                       method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # randomly cropping to 256 x 256 x 3
    # image = tf.image.random_crop(image, size=[IMG_HEIGHT, IMG_WIDTH, 3])
    # random mirroring
    image = tf.image.random_flip_left_right(image)
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    # image = tf.image.resize(images=image, size=[IMG_WIDTH, IMG_WIDTH])
    # image = image/255 #[0 1]
    return image

def data_generator(image_list,train=False):
    dataset = tf.data.Dataset.from_tensor_slices(image_list)
    dataset = dataset.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [ ]:
# dataset, metadata = tfds.load('cycle_gan/horse2zebra',
#                               with_info=True, as_supervised=True)
#
# train_horses, train_zebras = dataset['trainA'], dataset['trainB']
# test_horses, test_zebras = dataset['testA'], dataset['testB']

In [ ]:
DATA_DIR = r"\\shelter\Kyu\unstain2stain\tiles\registrated_tiles"

In [ ]:

train_he = sorted(glob.glob(os.path.join(*[DATA_DIR, 'HE\*\*.png'])))
train_unstained = sorted(glob.glob(os.path.join(*[DATA_DIR,'Unstained\*\*.png'])))
val_he = sorted(glob.glob(os.path.join(*[DATA_DIR, 'HE\OTS_14684_3_he\*.png'])))
val_unstained = sorted(glob.glob(os.path.join(*[DATA_DIR, 'Unstained\OTS_14684_3\*.png'])))

In [ ]:
train_horses = data_generator(train_unstained) #unstained
train_zebras = data_generator(train_he) #he

In [ ]:


#
# # normalizing the images to [-1, 1]
# def normalize(image):
#   image = tf.cast(image, tf.float32)
#   image = (image / 127.5) - 1
#   return image
#
# def random_jitter(image):
#   # resizing to 286 x 286 x 3
#   image = tf.image.resize(image, [286, 286],
#                           method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#   # randomly cropping to 256 x 256 x 3
#   image = random_crop(image)
#   # random mirroring
#   image = tf.image.random_flip_left_right(image)
#   return image
#
# def preprocess_image_train(image,label):
#   image = random_jitter(image)
#   image = normalize(image)
#   return image
#
# def preprocess_image_test(image,label):
#   image = normalize(image)
#   return image

In [ ]:
# train_horses = train_horses.cache().map(
#     preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
#     BUFFER_SIZE).batch(BATCH_SIZE)
#
# train_zebras = train_zebras.cache().map(
#     preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
#     BUFFER_SIZE).batch(BATCH_SIZE)

# test_horses = test_horses.map(
#     preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
#     BUFFER_SIZE).batch(BATCH_SIZE)
#
# test_zebras = test_zebras.map(
#     preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
#     BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
sample_horse = next(iter(train_horses))
# sample_zebra = next(iter(train_zebras))

In [ ]:
# plt.subplot(121)
# plt.title('Horse')
# plt.imshow(sample_horse[0] * 0.5 + 0.5)
#
# plt.subplot(122)
# plt.title('Horse with random jitter')
# plt.imshow(random_jitter(sample_horse[0]) * 0.5 + 0.5)

In [ ]:
# plt.subplot(121)
# plt.title('Zebra')
# plt.imshow(sample_zebra[0] * 0.5 + 0.5)
#
# plt.subplot(122)
# plt.title('Zebra with random jitter')
# plt.imshow(random_jitter(sample_zebra[0]) * 0.5 + 0.5)

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [ ]:
# to_zebra = generator_g(sample_horse)
# to_horse = generator_f(sample_zebra)
# plt.figure(figsize=(8, 8))
# contrast = 8
#
# imgs = [sample_horse, to_zebra, sample_zebra, to_horse]
# title = ['Horse', 'To Zebra', 'Zebra', 'To Horse']
#
# for i in range(len(imgs)):
#   plt.subplot(2, 2, i+1)
#   plt.title(title[i])
#   if i % 2 == 0:
#     plt.imshow(imgs[i][0] * 0.5 + 0.5)
#   else:
#     plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)
# plt.show()

In [ ]:
# plt.figure(figsize=(8, 8))
#
# plt.subplot(121)
# plt.title('Is a real zebra?')
# plt.imshow(discriminator_y(sample_zebra)[0, ..., -1], cmap='RdBu_r')
#
# plt.subplot(122)
# plt.title('Is a real horse?')
# plt.imshow(discriminator_x(sample_horse)[0, ..., -1], cmap='RdBu_r')
#
# plt.show()

In [ ]:
LAMBDA = 10 #1 to 10.
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))

  return LAMBDA * loss1

def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

lr_gg = 2e-4
lr_gf = 2e-4
lr_dx = 2e-4
lr_dy = 2e-4

generator_g_optimizer = tf.keras.optimizers.Adam(lr_gg, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(lr_gf, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(lr_dx, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(lr_dy, beta_1=0.5)

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [ ]:
# This block trains new CycleGAN model
import PIL
EPOCHS = 100




In [ ]:
def tensor_to_image(tensor):
    # tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)


def generate_images(model, test_input):
  prediction = model(test_input)

  #fn = test_unstained[it][-8:]
  #it = it+1
  #Image.save(tensor_to_image(prediction[0]),r'\\shelter\Kyu\unstain2stain\ashley_pilot\dataset\syn_he\{}'.format(fn))

  plt.figure(figsize=(12, 12))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()
  return

In [ ]:

@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.

    fake_y = generator_g(real_x, training=True)
    cycled_x = generator_f(fake_y, training=True)

    fake_x = generator_f(real_y, training=True)
    cycled_y = generator_g(fake_x, training=True)

    # same_x and same_y are used for identity loss.
    same_x = generator_f(real_x, training=True)
    same_y = generator_g(real_y, training=True)

    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # calculate the loss
    gen_g_loss = generator_loss(disc_fake_y)
    gen_f_loss = generator_loss(disc_fake_x)

    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)

    # Total generator loss = adversarial loss + cycle loss
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)

  # Calculate the gradients for generator and discriminator
  generator_g_gradients = tape.gradient(total_gen_g_loss,
                                        generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss,
                                        generator_f.trainable_variables)

  discriminator_x_gradients = tape.gradient(disc_x_loss,
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss,
                                            discriminator_y.trainable_variables)

  # Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients,
                                            generator_g.trainable_variables))

  generator_f_optimizer.apply_gradients(zip(generator_f_gradients,
                                            generator_f.trainable_variables))

  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))

  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_horses, train_zebras)):
    train_step(image_x, image_y)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_horse)

  if (((epoch + 1) % 5 == 0) or (epoch == EPOCHS)):
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
#load checkpoint, predict another set of image, evaluate performance

In [46]:
#load new data
new_dir = r'\\shelter\Kyu\unstain2stain\ashley_pilot\dataset'
test_unstained = sorted(glob.glob(os.path.join(*[new_dir, 'us\*.png'])))
val_he = sorted(glob.glob(os.path.join(*[new_dir, 'he\*.png'])))

In [43]:
test_unstained[0]

'\\\\shelter\\Kyu\\unstain2stain\\ashley_pilot\\dataset\\us\\85481_45916xy0317.png'

In [47]:
#load test set and validation set
test_set = data_generator(test_unstained)

In [ ]:
#load test_set into model
it = 0
for inp in test_set.take(21):
  generate_images(generator_g, inp)


In [48]:
for inp in test_set.take(1):
    prediction = generator_g(inp)

In [49]:
import numpy as np
image1_numpy = prediction.numpy()

In [50]:
import cv2
im=cv2.normalize(image1_numpy[0],None,0,255,cv2.NORM_MINMAX, dtype=cv2.CV_8U)

In [51]:
from PIL import Image
dst = r'\\shelter\Kyu\unstain2stain\ashley_pilot\dataset\syn_he'
fn = '1.png'
Image.fromarray(im).save(os.path.join(dst,fn))

In [ ]:
#save generated data in new

In [ ]:
#image metrics

In [ ]:
import math
import numpy as np
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [ ]:
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [ ]:
#from vif_utils import vif
#print(vif(img_ref, img_dist))